# Data, Databases, SQL, and interacting with databases using Python

For this exercise, we will be using data contained in the "homework" database on the Big Data for Social Science Class Server. This notebook will walk you through accessing the class data using IPython Notebook and while familiarizing you with the available class data.

## Table of Contents

- [Tables we will look at](#Tables-we-will-look-at)

    - [StarMetrics Database](#StarMetrics-Database)
    - [UMETRICS Grants Database](#UMETRICS-Grants-Database)
    - [USPTO Patents](#USPTO-Patents)

- [Databases and Python](#Databases-and-Python)

    - [Making and using a database connection](#Making-and-using-a-database-connection)
    
- [SQL](#SQL)

    - [Querying the database](#Querying-the-database)
    - [**Exercise 1**](#Exercise-1)
    - [WHERE clauses: Limiting the results](#WHERE-clauses:-Limiting-the-results)
    - [**Exercise 2**](#Exercise-2)
    - [JOIN: Connecting multiple tables](#JOIN:-Connecting-multiple-tables)
    - [GROUP BY and aggregate functions](#GROUP-BY-and-aggregate-functions)
    - [**Exercise 3**](#Exercise-3)
    - [ORDER BY](#ORDER-BY)
    - [Modifying the database](#Modifying-the-database)
    - [**Exercise 4**](#Exercise-4)

- [References](#References)

## Tables we will look at

- Back to the [Table of Contents](#Table-of-Contents)

For these exercises we will use tables in the homework database. These tables were created using data from the starmetrics, umetricsgrants and usptopatents databases. Each of these databases contain different types of information and are available for your use during this class. Also available to you, is your own database. These databases have the same name as your username. 

Quick description of the data available: 

### StarMetrics Database

- Back to the [Table of Contents](#Table-of-Contents)

The starmetrics database contains transactional data from universities that describe expenditures on federal research grants. The data includes four different types of expenditures: 

- 1) employee expenditures - this describes the people by occupation who charged time to federal grants.
- 2) vendor expenditures - this descirbes the businesses that goods were bought from federal grants.
- 3) subaward expenditures - this describes the universities and other institutions that are paid to collaborate from federal grants.
- 4) award expenditures - this describes the overhead that is associated with each federal grant.

### UMETRICS Grants Database 

- Back to the [Table of Contents](#Table-of-Contents)

The umetricsgrants database contains public data that describes NIH, NSF, USDA & NASA federal awards. This database was created by combining several small databases together to capture all the grant data in one database. The structure of the database tables are different depending on the source of the data.

### USPTO Patents

- Back to the [Table of Contents](#Table-of-Contents)

The USPTOPatents database contains public patent and inventor data. These data include all patents, inventors, assignees and their associated metadata on location, patent classes, etc. 

## Databases and Python

- Back to the [Table of Contents](#Table-of-Contents)

### Making and using a database connection

- Back to the [Table of Contents](#Table-of-Contents)

Python lets you interact with databases using SQL just like you would in any SQL gui or terminal. Python code can do SELECTs, CREATEs, INSERTs, UPDATEs, and DELETEs, and any other SQL, and the results are returned in a format that lets you interact with them after the SQL statements finish.

To interact with a database using python, first you have to connect to the database.

To create a database connection, you first must import that database's DB-API implementation, then you call the connect() function, passing it information on where to find the database to which you are trying to connect.

In [ ]:
# imports
import MySQLdb

# declare variables
user = "username"
passwd = "password"

db = MySQLdb.connect(user=user, passwd=passwd, db="homework")

Next, you use the connection to create a cursor. A cursor takes SQL written in python, delivers it to the database. It then uses the results and converts them to a format that can be interacted with using python, and returns that transformed, usable response back to you.

To make a cursor, call the cursor() method on the connection object instance returned by the call to connect.

In [ ]:
cur = db.cursor(MySQLdb.cursors.DictCursor)

To execute SQL, just pass the SQL statement to the cursor's execute() method and store the result in a variable. 

To return only the list of variables that are contained in the table - use cur.fetchone()

For example: 

In [ ]:
cur.execute("SELECT * FROM OSU_vendor")

list(cur.fetchone())

To return a set number of rows that are contained in the table - use cur.fetchmany(size=#)

In [ ]:
cur.execute("SELECT * FROM OSU_vendor")

list(cur.fetchmany(size=1))

To return the full table - use cur.fetchall()

Be careful, this can take awhile depending on how big the table is! Before we fetchall for table homework.OSU_vendor, we will count how many rows are in the table.

In [ ]:
cur.execute("SELECT count(*) FROM OSU_vendor")

list(cur.fetchall())

In [ ]:
cur.execute("SELECT * FROM OSU_vendor")

list(cur.fetchall())

And lastly, to close the database connection, we do the following: 

In [ ]:
# close cursor
cur.close()
    
# close connection
db.close()

## SQL

- Back to the [Table of Contents](#Table-of-Contents)

Now that we know how to make a connection to a database using Ipython, we can begin to master some SQL basics to help you get started with understanding the data and databases available. 

SQL is a quirky language. It is different from procedural languages like Python and is designed for a very specific purpose: to interact with relational data. It isn't structured like other languages, and while it can make data access easy, it also can make tasks that would be easy in other languages (though perhaps not exceptionally performant) confoundingly complex.  Let's dive in so you can see it for yourself!

### Querying the database

- Back to the [Table of Contents](#Table-of-Contents)

The basic method of querying the database is to use a select statement:

    SELECT *
    FROM OSU_vendor; 

Where:

- Columns or variables that would like returned are put in the SELECT clause (after the word "SELECT" but before the word "FROM").  An asterisk ( "\*" ) is a wildcard - it will return all columns for a given table.
- The name of the table (or names of the tables - more on this in a bit) you want to query is put after the word "FROM", in the FROM clause.
- It is considered good style to capitalize words in an SQL query that are SQL words, not variables, table names, or values you are filtering on or searching for, ie. SELECT, FROM, WHERE, etc.
- Although it isn't always necessary in MySQL, you should end SQL statements with a semi-colon.  It isn't required everywhere, but it is required in some contexts, so better to be aware and get into the habit.

Instead of specifying “all” columns ( "\*" ), you can specify which columns you want by name, in a comma-delimited list after "SELECT":

    SELECT uniqueawardnumber, fipscode, paymentamount
    FROM OSU_vendor;

You can specify calculations in the list of columns also:

    SELECT uniqueawardnumber, ( periodenddate - periodstartdate + 1 )
    FROM OSU_vendor;

And you can give those new columns names:

    SELECT uniqueawardnumber, ( periodenddate - periodstartdate + 1 ) AS num_days
    FROM OSU_vendor;
    
You can also use special keywords and functions in the SELECT clause.  For example, the keyword "DISTINCT", which only returns any given value in a given column once:

    SELECT DISTINCT uniqueawardnumber
    FROM OSU_vendor;
    
And "COUNT()", which returns a count of matching rows rather than a list:
    
    SELECT COUNT( DISTINCT uniqueawardnumber )
    FROM OSU_vendor;

### Exercise 1

- Back to the [Table of Contents](#Table-of-Contents)

In [ ]:
#Create connection to the "homework" database "OSU_vendor" and "OSU_grant" table to answer the following questions: 

#Create Connection Here: 
user =
passwrd = 
db =
cur =

#1) How many distinct vendors are in the OSU vendor data?

cur.execute()
list(cur.fetchall()) 

#2) How many distinct topics are in the OSU grant data? 

cur.execute()
list(cur.fetchall())

#Close Connection Here: 


### WHERE clauses: Limiting the results

- Back to the [Table of Contents](#Table-of-Contents)

In a SELECT query, you can add a WHERE clause to limit the results:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30';

Where:

- you are making conditional tests, just like in a Python "if" statement.
- EXCEPT here, instead of "==" being the equality operator, it is just "=".
- Comparison operators:

    - "**_`=`_**" - equal to
    - "**_`!=`_**" or "**_`<>`_**" - not equal to
    - "**_`<`_**" - less than
    - "**_`<=`_**" - less-than-or-equal-to
    - "**_`>`_**" - greater than
    - "**_`>=`_**" - greater-than-or-equal-to
    - "**_`LIKE`_**" and "**_`NOT LIKE`_**" - wild-card matching operator, where percent matches 0 or more characters ( "%" ) and an underscore matches any 1 character ( "_" ).
    - "**_`IN( value_list )`_**" and "**_`NOT IN( value_list )`_**" - checks whether the value to the left of the "IN", usually a column's value in a given row, is either IN or NOT IN the list on the right of the IN.
    
An example of using LIKE:

    SELECT *
    FROM OSU_vendor
    WHERE uniqueawardnumber LIKE '%EY022601%'

You can specify multiple conditions for matching in your WHERE clauses, as well, to more precisely filter the results of your query:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    
Note:

- when you are matching a column whose type is numeric, you just put the value in the query, with no quotation marks (just like in Python).
- when you are filtering a string column, you have to include the value you are looking for (the value on the right-hand side of the equal sign) in single-quotes. They must be single-quotes, too.  Unlike in Python, double-quotes have an entirely different meaning that single quotes in SQL, and can cause your query to fail.

Like "None" in Python, the signifier of an unset value in a column for a row is special - NULL.  To check for NULL, you use "IS NULL" or "IS NOT NULL", rather than the "=" or "!=".

    /* find missing values */
    SELECT *
    FROM OSU_vendor
    WHERE institutionid IS NULL;

You can also explicitly cut off the number of results your query returns using the LIMIT keyword.  Just LIMITing to 10 only returns the first 10 results for the query:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    LIMIT 10;
    
You can also use LIMIT to skip to the middle of the results by giving it two numbers, separated by a comma.  The first number is the number of records you want to skip, the second number is how many records you want to include after you skip:

    /* skip 10, the output 15 */
    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    LIMIT 10, 15;

### Exercise 2 

- Back to the [Table of Contents](#Table-of-Contents)

In [ ]:
#Create connection to the "homework" database "OSU_vendor" and "OSU_grant" table to answer the following questions: 

#Create Connection Here: 
user =
passwrd = 
db =
cur =

#1) What is the text that describes Topic 45? 

cur.execute()
list(cur.fetchall()) 

#2) How well does topic ID 45 fit (percentage) to award number "1115005"? 

cur.execute()
list(cur.fetchall())

#Close Connection Here: 

### JOIN: Connecting multiple tables

- Back to the [Table of Contents](#Table-of-Contents)

We can specify multiple tables in the FROM clause of a select query. This is called a “join”. However, when we do, we need to remember to specify how to match up rows across the two tables. Usually, there is a column that is the same in both tables that can be used to match them up. For much of the starmetrics database, that would be a column like uniqueawardnumber or award_id. 

Also, we frequently give tables temporary short names to make it easy to refer to them.

    /* Lists the topics associated with each federal award */
    SELECT DISTINCT v.uniqueawardnumber, g.topic_text
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber;

We can still use regular WHERE clauses in these queries, too, to further filter:

    /* Lists the topics for each federal NSF award in 2012 */
    SELECT DISTINCT v.uniqueawardnumber, g.topic_text
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber
        AND agency_abbrev = "NSF"
        AND year(periodstartdate) = 2012;

Table joins are the most important feature of SQL databases; they are very powerful and allow us to create all kinds of complex queries. You can also join more than two tables if you like.

### GROUP BY and aggregate functions

- Back to the [Table of Contents](#Table-of-Contents)

Often, one thing that you want to do is to aggregate over multiple rows. For example, "What is the total expenditures for each award in 2012?" To do this, use a GROUP BY clause:

    /* sum vendor expenditures by award and filter by 2012 */
    SELECT uniqueawardnumber, SUM(paymentamount)
    FROM OSU_vendor
    WHERE year(periodstartdate) = 2012
    GROUP BY uniqueawardnumber;

There are a number of useful aggregate functions:

- **_SUM(column)_** : Calculate the sum of column for all the rows in each group
- **_AVG(column)_** : Calculate the numeric average for all of the rows in each group
- **_COUNT(column)_** : Count the number of rows in each group
- **_MIN(column) and MAX(column)_** : Find the minimum or maximum value of column in all the rows in each group

Often, it can be very powerful to combine GROUP BY and table joins. To figure out these queries, I recommend first getting the join to return the individual rows correctly, and then adding in the GROUP BY and aggregates.

### Exercise 3 

- Back to the [Table of Contents](#Table-of-Contents)

In [ ]:
#Create connection to the "homework" database "OSU_vendor" and "OSU_grant" table to answer the following questions: 

#Create Connection Here: 
user =
passwrd = 
db =
cur =

#1) What are the total expenditures in 2012? 

cur.execute()
list(cur.fetchall()) 

#2) What are the total expenditures in 2012 on NSF grants?  

cur.execute()
list(cur.fetchall())

#) What are the total expenditures in 2012 on NSF grants with topic ID 45?

cur.execute()
list(cur.fetchall())

#Close Connection Here: 


### ORDER BY

- Back to the [Table of Contents](#Table-of-Contents)

Normally, the results are returned in the order they appear in the database. However, it can be very useful to re-order the results using ORDER BY:

    SELECT uniqueawardnumber, paymentamount
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber
        AND v.university = g.university
    ORDER BY g.award_id

(After you specify which column to order by, you can optionally specify either ASC for ascending order, or DESC for descending order.)

Using ORDER BY with custom column names can be really useful when combined with GROUP BY:

    SELECT uniqueawardnumber, SUM(paymentamount)
    FROM OSU_vendor
    WHERE year(periodstartdate) = 2012
    GROUP BY uniqueawardnumber
    ORDER BY sum(paymentamount) DESC

### Modifying the database

- Back to the [Table of Contents](#Table-of-Contents)

In addition to retrieving information from an existing database, you can also insert data into a database, update existing rows, and delete records using SQL.  Permissions on the homework, starmetrics, umetricsgrants and usptopatents databases will not allow you to modify the databases. For these exercises, we open an additional connection to your individual user database. 

Here are some example queries:

- **CREATE**: Adding a table to a database

        CREATE TABLE cjones.data (
        ID int(11) auto_increment primary key, 
        name_first varchar(20)
        name_last varchar(30))

- **INSERT**: Adding a row to a table

        INSERT INTO cjones.data
        (name_first, name_last)
        VALUES ('Christina', 'Jones')

- **UPDATE**: Changing data that is already in a table

        UPDATE cjones.data
        SET name_last = 'Johnson'
        WHERE name_first = 'Christina'
        
- **ALTER TABLE**: Changing the structure of an existing table

        ALTER TABLE cjones.data
            ADD COLUMN gender VARCHAR(1) DEFAULT 'F'

- **DELETE**: Removing one or more rows from a table

        DELETE FROM cjones.data
        WHERE name_last = 'johnson'

- **DELETE**: removing table from database

        DELETE cjones.data


Lastly, you can also CREATE a table using an existing table. 

- **CREATE**: Adding a table to a database

        CREATE TABLE cjones.osu_vendor (
        SELECT * FROM homework.OSU_vendor
        WHERE year(periodstartdate) = 2012
        and agency_abbrev = 'NSF');

### Exercise 4 

- Back to the [Table of Contents](#Table-of-Contents)

In [ ]:
#Create connection to the "homework" database "OSU_vendor" and "OSU_grant" tables and to your own database create and modify your own tables.


#Create Connection Here: 
user =
passwrd = 
db =
cur =

#1) Create a table in your home database with the following parameters - all vendor expenditures on 6-30-2014 with topic ID 45. Use name "OSU_exp_on_t45"

cur.execute()
list(cur.fetchall()) 

#Close Connection Here: 


## References 

- Back to the [Table of Contents](#Table-of-Contents)

The tables for this homework were created directly from the starmetrics and umetricsgrants databases. The following SQL code was used to generate the tables: 

In [ ]:
CREATE TABLE homework.osu_vendor (
    SELECT periodstartdate, periodenddate, v.uniqueawardnumber, recipientaccountnumber, institutionid, paymentamount, v.university, v.cfda, 
        v.zipcode, fipscode, statecode, countycode, c.agency, agency_abbrev, agency_text, sub_agency_text, program_title
    FROM starmetrics.vendor v
        LEFT JOIN starmetrics.zip_to_fip z on z.zipcode = v.zipcode
        LEFT JOIN starmetrics.cfda c on c.cfda = v.cfda
    WHERE v.university = "OSU"
        AND periodstartdate >= "2011-01-01" AND v.zipcode != "" )

In [ ]:
CREATE TABLE homework.OSU_grant (
    SELECT award_id, topic_id, model, application_id, proportion, seq, agency, topic_text, uniqueawardnumber, university 
    FROM umetricsgrants.topiclda t
        LEFT JOIN umetricsgrants.topiclda_text text using(topic_id, model)
        LEFT JOIN starmetrics.crosswalk c using(award_id) 
    WHERE t.model = "NSF"
        AND seq = 1
        AND university = "OSU")